In [1]:
from dolfinx import fem, mesh, plot, io, geometry
from dolfinx.fem.petsc import LinearProblem
from petsc4py.PETSc import ScalarType
from mpi4py import MPI
import numpy as np
from ufl import ds, dx, grad, inner
import ufl
import matplotlib as mpl
import pyvista

import sys
import os
# Get the absolute path to the src directory
src_path = os.path.abspath('..')
if src_path not in sys.path:
    sys.path.append(src_path)

from typing import List, Tuple
from src.visualization import timeDependentVariableToGif, plot_array, plot_function
from src.solveStateEquation import solveStateEquation, getSourceTerm
from src.solveAdjointEquation import solveAdjointEquation
from src.solveFinDimObjective import solveFinDimProblem
from src.tools import getValueOfFunction, buildIterationFunction
from src.ExtremalPoints import ExtremalPoint
from src.HesseMatrix import calculateL2InnerProduct, HesseMatrix
from src.semiSmoothNewtonSolver import computeSemiNewtonStep
from dataclasses import dataclass

@dataclass
class Parameters:
    T = 1
    dt = 0.01
    x1 = (0.5, 0.5)
    x2 = (-0.5, -0.5)
    area = 4
    d = 2
    mollify_const = 0.01
    newton_c = 1
    eta = 0.01
    alpha = 2.2
    beta = 2.4
    waveSpeed = 1
    randomFactor = 0.05
    solverIteration = 0
    yd = []
    msh = mesh.create_rectangle(
        comm=MPI.COMM_WORLD,
        points=((-1., -1.), (1.0, 1.0)),
        n=(32, 32),
        cell_type=mesh.CellType.triangle,
    )
    V = fem.functionspace(msh, ("Lagrange", 1))

['/mnt/c/Users/Thomas/repos/TGV-regularized-wave/python/test', '/home/thomas/miniconda3/envs/fenicsx-env/lib/python312.zip', '/home/thomas/miniconda3/envs/fenicsx-env/lib/python3.12', '/home/thomas/miniconda3/envs/fenicsx-env/lib/python3.12/lib-dynload', '', '/home/thomas/miniconda3/envs/fenicsx-env/lib/python3.12/site-packages']


# Test linearity property of solver

In [12]:
# Test stability of solver for under addition
params = Parameters()

ext1 = ExtremalPoint(np.array([1.0, 0]), 0.5, type=0, params=params, idx=int(0.5/params.dt))
ext2 = ExtremalPoint(np.array([1.0, 0]), 0.2, type=1, params=params, idx=int(0.7/params.dt))
g1 = getSourceTerm(params.x1, params)
g2 = getSourceTerm(params.x2, params)
factor = 2

active_set = []
active_set.append(ext1)
weights = np.ones(len(active_set))
slope = np.ones((params.d,))
y_shift = np.ones((params.d,))
u_1 = lambda t: buildIterationFunction(t, active_set, weights, slope, y_shift, params)[0]
u_2 = lambda t: buildIterationFunction(t, active_set, weights, slope, y_shift, params)[1]
K_u1, control_u1 = solveStateEquation([g1, g2], [u_1, u_2], params)

active_set = []
active_set.append(ext2)
weights = factor * np.ones(len(active_set))
slope = np.ones((params.d,))
y_shift = np.ones((params.d,))
u_1 = lambda t: buildIterationFunction(t, active_set, weights, slope, y_shift, params)[0]
u_2 = lambda t: buildIterationFunction(t, active_set, weights, slope, y_shift, params)[1]
K_u2, control_u2 = solveStateEquation([g1, g2], [u_1, u_2], params)

active_set = []
active_set.append(ext1)
active_set.append(ext2)
weights = np.ones(len(active_set))
weights[1] = factor
slope = 2 * np.ones((params.d,))
y_shift = 2 * np.ones((params.d,))
u_1 = lambda t: buildIterationFunction(t, active_set, weights, slope, y_shift, params)[0]
u_2 = lambda t: buildIterationFunction(t, active_set, weights, slope, y_shift, params)[1]
K_u_sum, control_u_sum = solveStateEquation([g1, g2], [u_1, u_2], params)

sum = []
max_diff = np.zeros(len(K_u1))
for idx in range(len(K_u1)):
    func = fem.Function(params.V)
    func.x.array[:] = K_u1[idx].x.array + K_u2[idx].x.array
    sum.append(func)
    max_diff[idx] = np.max(np.abs(func.x.array - K_u_sum[idx].x.array))
print('Maximal difference in addition: ', np.max(max_diff))

Maximal difference in addition:  4.9404924595819466e-15


# Test linearity of adjoint solver

In [13]:
ext1 = ExtremalPoint(np.array([1.0, 0]), 0.5, type=0, params=params, idx=int(0.5/params.dt))
ext2 = ExtremalPoint(np.array([1.0, 0]), 0.2, type=1, params=params, idx=int(0.7/params.dt))
g1 = getSourceTerm(params.x1, params)
g2 = getSourceTerm(params.x2, params)
factor = 2

active_set = []
active_set.append(ext1)
weights = np.ones(len(active_set))
slope = np.ones((params.d,))
y_shift = np.ones((params.d,))
u_1 = lambda t: buildIterationFunction(t, active_set, weights, slope, y_shift, params)[0]
u_2 = lambda t: buildIterationFunction(t, active_set, weights, slope, y_shift, params)[1]
K_u1, control_u1 = solveStateEquation([g1, g2], [u_1, u_2], params)

active_set = []
active_set.append(ext2)
weights = factor * np.ones(len(active_set))
slope = np.ones((params.d,))
y_shift = np.ones((params.d,))
u_1 = lambda t: buildIterationFunction(t, active_set, weights, slope, y_shift, params)[0]
u_2 = lambda t: buildIterationFunction(t, active_set, weights, slope, y_shift, params)[1]
K_u2, control_u2 = solveStateEquation([g1, g2], [u_1, u_2], params)

active_set = []
active_set.append(ext1)
active_set.append(ext2)
weights = np.ones(len(active_set))
weights[1] = factor
slope = 2 * np.ones((params.d,))
y_shift = 2 * np.ones((params.d,))
u_1 = lambda t: buildIterationFunction(t, active_set, weights, slope, y_shift, params)[0]
u_2 = lambda t: buildIterationFunction(t, active_set, weights, slope, y_shift, params)[1]
K_u_sum, control_u_sum = solveStateEquation([g1, g2], [u_1, u_2], params)

K_u_sum_adjoint = solveAdjointEquation(K_u_sum, params)
K_u_1_adjoint = solveAdjointEquation(K_u1, params)
K_u_2_adjoint = solveAdjointEquation(K_u2, params)

sum = []
max_diff = np.zeros(len(K_u1))
for idx in range(len(K_u1)):
    func = fem.Function(params.V)
    func.x.array[:] = K_u_1_adjoint[idx].x.array + K_u_2_adjoint[idx].x.array
    sum.append(func)
    max_diff[idx] = np.max(np.abs(func.x.array - K_u_sum_adjoint[idx].x.array))
print('Maximal difference in addition: ', np.max(max_diff))

Maximal difference in addition:  9.853229343548264e-16
